In [1]:
import ray

In [2]:
ray.shutdown()

In [3]:
ray.init()

2021-05-19 15:17:10,509	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '9.163.5.112',
 'raylet_ip_address': '9.163.5.112',
 'redis_address': '9.163.5.112:6379',
 'object_store_address': '/tmp/ray/session_2021-05-19_15-17-09_977363_85364/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-19_15-17-09_977363_85364/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-19_15-17-09_977363_85364',
 'metrics_export_port': 58731,
 'node_id': '47cffbd91e125383878344e76dd07babe812da2b'}

In [4]:
%config IPCompleter.use_jedi = False

In [5]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler

transformer = FunctionTransformer(np.log1p)
minmax_scaler = MinMaxScaler()

In [6]:
import codeflare.pipelines.Datamodel as dm

class FeatureUnion(dm.AndFunc):
    def __init__(self):
        pass
    
    def eval(self, xy_list):
        X_list = []
        y_list = []
        
        for xy in xy_list:
            X_list.append(xy.get_x())
        X_concat = np.concatenate(X_list, axis=0)
        
        return Xy(X_concat, None)

In [7]:
pipeline = dm.Pipeline()

node_a = dm.OrNode('a', minmax_scaler)
node_b = dm.OrNode('b', minmax_scaler)
node_c = dm.AndNode('c', FeatureUnion())
# node_d = dm.OrNode('d', transformer)

pipeline.add_edge(node_a, node_c)
pipeline.add_edge(node_b, node_c)
# pipeline.add_edge(node_b, node_d)

In [8]:
X = np.array([0, 1, 2, 3])
X = np.reshape(X, (4, 1))

In [9]:
from codeflare.pipelines.Datamodel import Xy
from codeflare.pipelines.Datamodel import XYRef

In [10]:
# Xy = XY(X, None)
# Xy_ref = ray.put(Xy)

X_ref = ray.put(X)
y_ref = ray.put(None)

Xy_ref = XYRef(X_ref, y_ref)
Xy_ref_ptr = ray.put(Xy_ref)
Xy_ref_ptrs = [Xy_ref_ptr]

In [11]:
import codeflare.pipelines.Runtime as rt

In [12]:
in_args={node_a: Xy_ref_ptrs, node_b: Xy_ref_ptrs}

In [13]:
from codeflare.pipelines.Runtime import ExecutionType

In [14]:
out_args = rt.execute_pipeline(pipeline, ExecutionType.FIT, in_args)

In [15]:
for key in out_args.keys():
    print(str(key))

c


In [16]:
out_Xyrefs = ray.get(out_args[node_c])

In [18]:
for out_xyref in out_Xyrefs:
    x = ray.get(out_xyref.get_Xref())
    and_func = ray.get(out_xyref.get_noderef()).get_and_func()
    print(x)
    print(and_func)

[[0.        ]
 [0.33333333]
 [0.66666667]
 [1.        ]
 [0.        ]
 [0.33333333]
 [0.66666667]
 [1.        ]]


(pid=gcs_server) F0519 16:14:23.850416 85373 158993920 gcs_node_manager.cc:528]  Check failed: _s.ok() Bad status: Disconnected: Redis is disconnected
(pid=gcs_server) *** Check failure stack trace: ***
(pid=gcs_server)     @        0x1024b8b72  google::LogMessage::~LogMessage()
(pid=gcs_server)     @        0x1024707d5  ray::RayLog::~RayLog()
(pid=gcs_server)     @        0x101ff2371  ray::gcs::GcsNodeManager::SendBatchedHeartbeat()
(pid=gcs_server)     @        0x102009d33  boost::asio::detail::binder1<>::operator()()
(pid=gcs_server)     @        0x102009971  boost::asio::detail::wait_handler<>::do_complete()
(pid=gcs_server)     @        0x102603a73  boost::asio::detail::scheduler::do_run_one()
(pid=gcs_server)     @        0x1025f78b2  boost::asio::detail::scheduler::run()
(pid=gcs_server)     @        0x1025f774b  boost::asio::io_context::run()
(pid=gcs_server)     @        0x101fa9b26  main
(pid=gcs_server)     @     0x7fff204d1f3d  start
(pid=gcs_server)     @                0x